In [1]:
import numpy as np
from glob import glob
from spec_id import Galaxy_ids
import pandas as pd
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import os
from shutil import copyfile
import seaborn as sea
from astropy.table import Table
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({"xtick.direction": "in","ytick.direction": "in"})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.1, as_cmap=True)

In [3]:
gal_id = Galaxy_ids('../data/masslist_mar22.dat')

In [8]:
gal_id.ids

array(['n14713', 'n16758', 'n21156', 'n21427', 'n23435', 'n32566',
       'n34694', 'n37686', 's35774', 's38785', 's39012', 's39170',
       's39241', 's39631', 's39804', 's39805', 's40597', 's40862',
       's41520', 's42221', 's44620', 's45972', 's46066', 's47677', 's48631'], 
      dtype='|S34')

In [4]:
def Remove_ns(gid):
    x = [int(U[1:]) for U in gid]
    return np.array(x)

def Get_rshift(table, ID):
    z = table['z_m1'][ID-1]
    return z

def Mag(band):
    magnitude=25-2.5*np.log10(band)
    return magnitude

In [3]:
uvjDB = pd.read_pickle('../data/uvj_db.pkl')
quvjDB = uvjDB[uvjDB['Q/S'].map(lambda x:x.startswith('q'))]
ids = Remove_ns(quvjDB.index)
smask = np.array([U.startswith('s') for U in quvjDB.index])
nmask = np.array([U.startswith('n') for U in quvjDB.index])

goodss = '../../../catalogs_for_CLEAR/goodss_3dhst.v4.1.cats/Eazy/goodss_3dhst.v4.1.zout'
scat = Table.read(goodss,format='ascii')

goodsn = '../../../catalogs_for_CLEAR/goodsn_3dhst.v4.1.cats/Eazy/goodsn_3dhst.v4.1.zout'
ncat = Table.read(goodsn,format='ascii')

s_z = np.array([Get_rshift(scat,U) for U in ids[smask]])
n_z = np.array([Get_rshift(ncat,U) for U in ids[nmask]])
z_tot = np.append(n_z,s_z)

In [6]:
qgalDB = pd.DataFrame({'gids' : quvjDB.index,'ids' : ids, 'redshift':z_tot})
# pd.to_pickle(qgalDB,'../data/quiescent_gal_DB.pkl')

In [11]:
qgalDB = pd.read_pickle('../data/quiescent_gal_DB.pkl')
uvjDB = pd.read_pickle('../data/uvj_db.pkl')
quvjDB = uvjDB[uvjDB['Q/S'].map(lambda x:x.startswith('q'))]

In [16]:
hd_path = '../../../../../Volumes/Vince_homedrive/Extractions/'

obv_dir =glob(hd_path + '*-*')

In [37]:
ID = int(quvjDB.index[3][1:])
if ID < 1000:
    strID = '00%s' % ID
if 1000 < ID < 10000:
    strID = '0%s' % ID
if 10000 <= ID:
    strID = '%s' % ID
print strID

02116


In [43]:
def Get_file_list(dirs,gid):
    field = gid[0]
    intID = int(gid[1:])
    if intID < 1000:
        ID = '00%s' % intID
    if 1000 < intID < 10000:
        ID = '0%s' % intID
    if 10000 <= intID:
        ID = '%s' % intID
        
    direct = '../../../../../Volumes/Vince_homedrive/Extractions/Quiescent_galaxies/%s' % gid
       
    one_d =np.array([])
    one_d_n =np.array([])
    for i in range(len(dirs)):
        if field == 's':
            pres = glob(dirs[i] + '/1D/FITS/' + '*S*_%s.*' % ID)
        else:
            pres = glob(dirs[i] + '/1D/FITS/' + '*N*_%s.*' % ID)

        if len(pres) >=1:
            one_d= np.append(one_d,pres)
            pres_n = pres[0].replace(dirs[i] + '/1D/FITS/','')
            one_d_n= np.append(one_d_n,pres_n)
       
    if len(one_d) >=1:
        if field == 's':
            two_d =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/*S*_%s.*' % ID)
            if len(two_d) >= 1:
                two_d_n = []
                for i in range(len(two_d)):
                    two_d_n.append(two_d[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/',''))
        else:
            two_d =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/*N*_%s.*' % ID)
            if len(two_d) >= 1:
                two_d_n = []
                for i in range(len(two_d)):
                    two_d_n.append(two_d[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/2D/PNG/',''))
                    
        if field == 's':
            spec =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/*S*_%s.*' % ID)
            if len(spec) >= 1:
                spec_n = []
                for i in range(len(spec)):
                    spec_n.append(spec[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/',''))    
        else:
            spec =glob('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/*N*_%s.*' % ID)
            if len(spec) >= 1:
                spec_n = []
                for i in range(len(spec)):
                    spec_n.append(spec[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/COMBINED/1D/FITS/','')) 
                    
        if not os.path.exists(direct):
            os.makedirs(direct)

        if len(two_d) >= 1:
            for i in range(len(two_d_n)):
                copyfile(two_d[i], direct + '/' + two_d_n[i])

        if len(spec) >= 1:
            for i in range(len(spec)):
                copyfile(spec[i], direct + '/' + spec_n[i])

        for i in range(len(one_d_n)):
            copyfile(one_d[i], direct + '/' + one_d_n[i])
                
        print 'in data'

In [40]:
Get_file_list(obv_dir,'n1184')

In [44]:
for i in range(len(quvjDB.index)):
    print quvjDB.index[i]
    Get_file_list(obv_dir,quvjDB.index[i])

n542


n1184
n2023
n2116
n2265
n2302
n2484
n2684
n2882


n2913
n2960
n3125
n3133
n3735
n3843
n4108


n4748
n4868
n4907
n6026
n6453
n8466


in data
n8912
in data
n8949
in data
n9056
in data
n9533
in data
n9582


...

s45775
in data
s45972


in data
s46066
in data
s46345


in data
s46377
s46562
s46846
in data
s46888


s47223
in data
s47276
s47677


in data
s47691
in data
s47873
s48631
in data
s49708


In [11]:
"""Get F105W mags"""
Hn = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodsn_3dhst.v4.3.cat',format='ascii')['f_F105W']
Hs = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodss_3dhst.v4.3.cat',format='ascii')['f_F105W']

In [35]:
"""Get ra dec"""
ran = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodsn_3dhst.v4.3.cat',format='ascii')['ra']
ras = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodss_3dhst.v4.3.cat',format='ascii')['ra']
decn = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodsn_3dhst.v4.3.cat',format='ascii')['dec']
decs = Table.read('../../../Dropbox/Quiescent_galaxies/Catalog_update/goodss_3dhst.v4.3.cat',format='ascii')['dec']

In [36]:
def Get_Hband_mag(gid,slist,nlist):
    field = gid[0]
    ID = int(gid[1:])
    
    if field == 's':
        H = Mag(slist[ID-1])

    if field == 'n':
        H = Mag(nlist[ID-1])

    return H

def Get_ra_dec(gid, sra, sdec, nra, ndec):
    field = gid[0]
    ID = int(gid[1:])
    
    if field == 's':
        ra = sra[ID-1]
        dec = sdec[ID-1]

    if field == 'n':
        ra = nra[ID-1]
        dec = ndec[ID-1]
        
    return ra,dec

In [15]:
Get_Hband_mag('s1',Hs,Hn)

nan

In [39]:
hmag = np.zeros(len(qgalDB['gids']))
ra = np.zeros(len(qgalDB['gids']))
dec = np.zeros(len(qgalDB['gids']))

for i in range(len(qgalDB['gids'])):
        print qgalDB['gids'][i]
        hmag[i] = Get_Hband_mag(qgalDB['gids'][i],Hs,Hn)
        ra[i],dec[i] = Get_ra_dec(qgalDB['gids'][i],ras,decs,ran,decn)

n542
n1184
n2023
n2116
n2265
n2302
n2484
n2684
n2882
n2913
n2960
n3125
n3133
n3735
n3843
n4108
n4748
n4868
n4907
n6026
n6453
n8466
n8912
n8949
n9056
n9533
n9582
n9623
n9692
n9861
n10338
n10539
n11004
n11626
n11814
n12173
n12406
n12739
n13041
n13117
n13718
n14140
n14346
n14713
n15359
n15439
n15529
n15616
n15786
n16758
n17070
n17436
n17746
n17969
n18000
n18241
n18838
n19263
n19442
n20432
n20851
n21156
n21427
n21724
n21810
n22027
n22633
n23026
n23254
n23435
n23498
n23624
n23857
n25662
n26350
n26470
n26796
n27185
n27600
n29714
n30801
n30813
n31022
n31207
n32000
n32562
n32566
n32691
n32841
n33093
n33263
n33675
n34241
n34326
n34580
n34630
n34694
n35099
n35208
n35299
n35493
n35835
n36530
n36751
n37107
n37189
n37210
n37686
n37765
n37896
n38126
n38142
n38225
s433
s1523
s1924
s2383
s2707
s3075
s3236
s3258
s4125
s4210
s4321
s4554
s4560
s5030
s5063
s5269
s5652
s6098
s6106
s6185
s6209
s6632
s6791
s7118
s7238
s7503
s9278
s9500
s9704
s10044
s10436
s11195
s11428
s11937
s12134
s12385
s12869
s13020
s130

In [40]:
qgalDB['hmag']=hmag
qgalDB['ra']=ra
qgalDB['dec']=dec

In [47]:
gal_dir = glob('../../../../../Volumes/Vince_homedrive/Extractions/Quiescent_galaxies/*')

In [48]:
gal_dir = [gal_dir[i].replace('../../../../../Volumes/Vince_homedrive/Extractions/Quiescent_galaxies/','') for i in range(len(gal_dir))]

In [49]:
present_flag = np.repeat(False,len(qgalDB['gids']))

In [50]:
for i in range(len(qgalDB['gids'])):
    if qgalDB['gids'][i] in gal_dir:
        present_flag[i] = True

In [51]:
qgalDB['in_data'] =present_flag

In [53]:
all_g_gals = list(qgalDB[qgalDB['in_data'] == True][qgalDB['agn'] == False]['gids'])

/Users/Vince.ec/anaconda2/envs/astroconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [56]:
print len(all_g_gals)

78


In [20]:
for i in range(len(gal_id.ids)):
    if gal_id.ids[i] not in all_g_gals:
        print gal_id.ids[i]

In [52]:
qgalDB

,gids,ids,redshift,hmag,ra,dec,agn,in_data
0,n542,542,1.213,22.333801,189.099532,62.115753,False,False
1,n1184,1184,1.209,21.823322,189.206958,62.129700,False,False
2,n2023,2023,1.680,22.941112,189.225357,62.139447,False,False
3,n2116,2116,1.018,22.415274,189.084282,62.141001,False,False
4,n2265,2265,1.104,21.974041,189.064897,62.142943,False,False
5,n2302,2302,0.978,20.364209,189.084527,62.145644,False,False
6,n2484,2484,1.051,22.246583,189.086981,62.144791,False,False
7,n2684,2684,1.007,21.371846,189.091908,62.147403,False,False
8,n2882,2882,1.139,22.651291,189.044900,62.149160,False,False
9,n2913,2913,1.108,23.474565,189.055170,62.147938,False,False


In [57]:
pd.to_pickle(qgalDB,'../data/quiescent_gal_DB.pkl')